In [1]:
%load_ext autoreload
%autoreload 2

## Imports

In [2]:
import sys
import os
import pickle
import random

import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt

from torch.optim import Adam
from sklearn import metrics
from collections import OrderedDict 

sys.path.append(os.path.abspath('..'))

import datasets
import utils.more_torch_functions as mtf
from utils.modules import Parallel, MaxLayer
from utils.custom_activations import StepActivation
from utils.custom_loss import AsymBCELoss
from utils.misc import train_model

seed = 2872
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)

SAVE_PATH = os.path.join(os.path.abspath('..'), "backup")
PKL_PATH = os.path.join(SAVE_PATH, "bdd")
PTH_PATH = os.path.join(SAVE_PATH, "nn")
METRIC_PATH = os.path.join(SAVE_PATH, "metrics")

## Load data

In [3]:
dataset = datasets.AdultDataset

np_x, np_y = dataset.get_dataset(balancing=True, discretizing=True, hot_encoding=True)
x_data, y_data = torch.Tensor(np_x), torch.Tensor(np_y)
input_size = x_data.size(1)
print(x_data.size())

torch.Size([15682, 113])


In [4]:
def eval_model(x_train, y_train, x_valid, y_valid, model, metrics_average="binary"):
    model.eval()
    pred_train = model(x_train).detach()
    pred_valid = model(x_valid).detach()

    p_train, r_train, f_train, _ = metrics.precision_recall_fscore_support(y_train, pred_train, beta=1, average=metrics_average, labels=[0,1])
    p_valid, r_valid, f_valid, _ = metrics.precision_recall_fscore_support(y_valid, pred_valid, beta=1, average=metrics_average, labels=[0,1])

    return f_train, p_train, r_train, f_valid, p_valid, r_valid

def train_eval_model(x_train, y_train, x_valid, y_valid, model, criterion, optimizer, epochs=50, metrics_average="binary"):
    train_model(x_train, y_train, model, criterion, optimizer, epochs)
    return eval_model(x_train, y_train, x_valid, y_valid, model, metrics_average)

def print_eval(x_train, y_train, x_valid, y_valid, model, criterion, optimizer):
    f_train, p_train, r_train, f_valid, p_valid, r_valid = train_eval_model(x_train, y_train, x_valid, y_valid, model, criterion, optimizer)
    print(
        f"{'':<15}{'Train':^15}{'Valid':^15}",
        f"{'F1 score':<15}{f_train:^15.3f}{f_valid:^15.3f}",
        f"{'Precision':<15}{p_train:^15.3f}{p_valid:^15.3f}",
        f"{'Rappel':<15}{r_train:^15.3f}{r_valid:^15.3f}",
        sep="\n",
    )

## Network

In [5]:
class nApxNet(nn.Module):
    def __init__(self, n, hl=3) -> None:
        super().__init__()

        self.n_apx = n
        self.net = nn.Sequential(OrderedDict([
            ('nets', Parallel(OrderedDict([
                (f'apx{i}', ApproxNet(hl)) for i in range(1, self.n_apx+1)
            ]))),
            ('or_', MaxLayer()),
        ]))


    def forward(self, input):
        return self.net(input)
    
    def add_apx(self, module):
        self.n_apx += 1
        self.net.nets.add_module(f'apx{self.n_apx}', module)
    
class ApproxNet(nn.Module):
    def __init__(self, hl1):
        super().__init__()
        self.nn = nn.Sequential(OrderedDict([
            ('l1', nn.Linear(input_size,hl1)),
            ('a1', StepActivation()),
            ('l2', nn.Linear(hl1,5)),
            ('a2', StepActivation()),
            ('l3', nn.Linear(5,1)),
            ('a3', StepActivation()),
        ]))        

    def forward(self, x):
        x = self.nn(x)

        return x
    
class Net(nn.Module):
    def __init__(self):
        super().__init__()

        hl1 = 50
        hl2 = 25

        self. nn = nn.Sequential(OrderedDict([
            ('l1', nn.Linear(input_size,hl1)),
            ('a1', nn.ReLU()),
            ('l2', nn.Linear(hl1,hl2)),
            ('a2', nn.ReLU()),
            ('l3', nn.Linear(hl2,1)),
            ('a3', StepActivation()),
        ]))
    
    def forward(self, x):
        x = self.nn(x)

        return x

## Exp

In [6]:
def relative_cov(x_train, y_train, x_valid, y_valid):
    exp_folder = "relative_cov"
    os.makedirs(os.path.join(PTH_PATH, exp_folder), exist_ok=True)
    # d_metrics = dict()
    epochs = 500
    
    model = nApxNet(1, hl=5)
    criterion = AsymBCELoss(1.2) # HERE
    module = model.net.nets.apx1
    optimizer = Adam(module.parameters(), lr=1e-2, weight_decay=1e-6)

    train_model(x_train, y_train, model, criterion, optimizer, epochs)

    mtf.freeze_model(model)

    module = Net()
    model.add_apx(module)
    criterion = nn.BCELoss() # CHANGE ???
    optimizer = Adam(module.parameters(), lr=1e-2, weight_decay=1e-6)

    train_model(x_train, y_train, model, criterion, optimizer, epochs)

    model.eval()
    global_pred_valid = model(x_valid).detach()
    apx_pred_valid = model.net.nets.apx1(x_valid).detach()
    nn_pred_valid = model.net.nets.apx2(x_valid).detach()

    glob_recall = metrics.recall_score(y_valid, global_pred_valid, pos_label=1, average='binary')
    apx_recall = metrics.recall_score(y_valid, apx_pred_valid, pos_label=1, average='binary')
    apx_misszero = 1 - metrics.precision_score(y_valid, apx_pred_valid, pos_label=1, average='binary')
    nn_recall = metrics.recall_score(y_valid, nn_pred_valid, pos_label=1, average='binary')
    relative_recall = apx_recall/glob_recall

    glob_prec = metrics.precision_score(y_valid, global_pred_valid, average='micro')

    net = Net()
    criterion = nn.BCELoss()
    optimizer = Adam(net.parameters(), lr=1e-2, weight_decay=1e-6)

    train_model(x_train, y_train, net, criterion, optimizer, epochs)

    net.eval()
    net_pred_valid = net(x_valid).detach()
      
    net_prec = metrics.precision_score(y_valid, net_pred_valid, average='micro')

    return glob_recall, apx_recall, apx_misszero, nn_recall, relative_recall, glob_prec, net_prec

In [7]:
max_iter = 100
sum = np.zeros(7) 
for _ in range(max_iter):
    train_index, valid_index = torch.utils.data.random_split(range(x_data.size(0)), [0.9, 0.1])

    x_train, y_train = x_data[train_index], y_data[train_index]
    x_valid, y_valid = x_data[valid_index], y_data[valid_index]

    res = relative_cov(x_train, y_train, x_valid, y_valid)
    res = np.array(res)
    sum+=res

sum/=max_iter
print(f"Model cov : {sum[0]:.3f}",
      f"NN cov : {sum[3]:.3f}",
      f"Approx cov : {sum[1]:.3f}",
      f"Approx misclassified zeros : {sum[2]:.3f}", # number of zeros predicted as one / number predicted as one
      f"Relative cov : {sum[4]:.3f}",
      f"Model precision : {sum[5]:.3f}",
      f"NN alone : {sum[6]:.3f}",
      sep='\n')

Model cov : 0.891
NN cov : 0.616
Approx cov : 0.839
Approx misclassified zeros : 0.260
Relative cov : 0.938
Model precision : 0.775
NN alone : 0.805


In [8]:
# def compute_metrics(pred_train, y_train, pred_valid, y_valid):
#     p_train, r_train, f_train, _ = metrics.precision_recall_fscore_support(y_train, pred_train, beta=1, average=None, labels=[0,1])
#     p_valid, r_valid, f_valid, _ = metrics.precision_recall_fscore_support(y_valid, pred_valid, beta=1, average=None, labels=[0,1])

#     metrics_list = [
#         f_train.mean(), f_valid.mean(),
#         p_train[0], p_valid[0],
#         r_train[0], r_valid[0],
#         p_train[1], p_valid[1],
#         r_train[1], r_valid[1],
#     ]

#     return metrics_list

# net = Net()
# crit = nn.BCELoss()
# opt = Adam(net.parameters(), lr=1e-2, weight_decay=1e-6)

# train_model(x_train, y_train, net, crit, opt, max_epoch=500)

# net_pred_train = net(x_train).detach().round()
# net_pred_valid = net(x_valid).detach().round()

# net_metrics = compute_metrics(net_pred_train, y_train, net_pred_valid, y_valid)

# print(net_metrics)